In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras.backend as K

Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train[:,:,:,np.newaxis] / 255.
x_test = x_test[:,:,:,np.newaxis] / 255.

In [3]:
L = L0 = keras.layers.Input((28,28,1))
L = keras.layers.Conv2D(4, (3,3), padding='same')(L)
L = keras.layers.BatchNormalization()(L)
L = keras.layers.LeakyReLU()(L)
L = keras.layers.MaxPool2D()(L)
L = keras.layers.Conv2D(8, (3,3), padding='same')(L)
L = keras.layers.BatchNormalization()(L)
L = keras.layers.LeakyReLU()(L)
L = keras.layers.MaxPool2D()(L)
L = keras.layers.Conv2D(16, (3,3), padding='same')(L)
L = keras.layers.BatchNormalization()(L)
L = keras.layers.LeakyReLU()(L)
L = keras.layers.MaxPool2D()(L)
L = keras.layers.Flatten()(L)

L_dyn = keras.layers.Lambda(lambda _: K.variable(np.random.randn(1, 3*3*16 * 10 + 1, 32)))([])
L_dyn = keras.layers.Lambda(lambda x: K.tile(x[1], [K.shape(x[0])[0],1,1]))([L, L_dyn])
L_dyn = keras.layers.Lambda(lambda x: K.concatenate([
    K.concatenate([
        K.repeat(x[0], 3*3*16 * 10),
        K.ones([K.shape(x[0])[0],1,3*3*16])], axis=-2),
    x[1]]))([L, L_dyn])
L_dyn = keras.layers.TimeDistributed(keras.layers.Dense(1))(L_dyn)
L_dyn = keras.layers.TimeDistributed(keras.layers.LeakyReLU())(L_dyn)
L_dyn_w = keras.layers.Lambda(lambda x: x[:,:-1])(L_dyn)
L_dyn_w = keras.layers.Reshape((10, 3*3*16))(L_dyn_w)
L_dyn_b = keras.layers.Lambda(lambda x: x[:,-1:])(L_dyn)
L_dyn_b = keras.layers.Reshape((1,))(L_dyn_b)

L = keras.layers.Lambda(lambda x: K.batch_dot(x[1], x[0]) + x[2])([L, L_dyn_w, L_dyn_b])
L = keras.layers.Softmax()(L)

M = keras.models.Model([L0], [L])
M.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])
M.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 4)    40          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 28, 28, 4)    16          conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 28, 28, 4)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [4]:
M.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 106s 2ms/step - loss: 0.4339 - acc: 0.8685 - val_loss: 0.1663 - val_acc: 0.9469
Epoch 2/10
60000/60000 [==============================] - 103s 2ms/step - loss: 0.1482 - acc: 0.9553 - val_loss: 0.1243 - val_acc: 0.9608
Epoch 3/10
60000/60000 [==============================] - 102s 2ms/step - loss: 0.1177 - acc: 0.9630 - val_loss: 0.0991 - val_acc: 0.9681
Epoch 4/10
60000/60000 [==============================] - 101s 2ms/step - loss: 0.1029 - acc: 0.9684 - val_loss: 0.0876 - val_acc: 0.9717
Epoch 5/10
60000/60000 [==============================] - 100s 2ms/step - loss: 0.0937 - acc: 0.9714 - val_loss: 0.0900 - val_acc: 0.9693
Epoch 6/10
60000/60000 [==============================] - 100s 2ms/step - loss: 0.0877 - acc: 0.9728 - val_loss: 0.0756 - val_acc: 0.9761
Epoch 7/10
60000/60000 [==============================] - 99s 2ms/step - loss: 0.0820 - acc: 0.9741 - val_loss: 0.0773 - v